<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libaries

In [55]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

# Load the data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Load the contract_classification.csv file
contract_class_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/contract_classification.csv'
contract_class = pd.read_csv(contract_class_path)



# Display the first few rows to verify content
print(contract_class.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-3-94d8358f3681>:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  contract_class = pd.read_csv(contract_class_path)


     ANO_SID CORPORATE_DEVISION   ORTPLZ  ORTS-NAME               STRASSE  \
0  4114028.0                VHV  42109.0  Wuppertal     Hans-Böckler-Str.   
1  4114039.0                VHV  42277.0  Wuppertal       Liegnitzer Str.   
2  4114045.0                VHV  42389.0  Wuppertal             Rascheweg   
3  4114049.0                VGV  42277.0  Wuppertal               Am Diek   
4  4114055.0                VHV  42553.0    Velbert  Emil-Schniewind-Str.   

   SUM_INSURED CONSTRACTION_DESIGN  CONSTRUCTION_YEAR        WFL ZONE  ...  \
0      71081.0      NORMAL_VENTURE        1967.565648   69.00000  2.0  ...   
1      55708.0      NORMAL_VENTURE        1967.565648   65.00000  4.0  ...   
2      74148.0      DESIGN_CLASS_I        1967.565648   75.00000  1.0  ...   
3     664000.0      NORMAL_VENTURE        1967.565648  106.24368  NaN  ...   
4      75682.0      NORMAL_VENTURE        1967.565648  119.00000  2.0  ...   

   DRAIN_PIPE_INSURED  PRODUCTLINE  PRIOR_DAMAGES UVV-KZ  UNDERWRITE

In [4]:
print(contract_class['DAMAGE'].value_counts())

DAMAGE
0    11389665
1      184774
Name: count, dtype: int64


# Models

## Preprocess the data

In [5]:
#Choose variables where it makes logical sense

initial_features = contract_class.loc[:, ['CORPORATE_DEVISION','ORTPLZ',  'SUM_INSURED', 'CONSTRACTION_DESIGN','CONSTRUCTION_YEAR','WFL','ZONE','DRAIN_PIPE_INSURED','PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER','DAMAGE']]

initial_features_clean = initial_features.dropna()

print(initial_features.info())

print(initial_features_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11574439 entries, 0 to 11574438
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   CORPORATE_DEVISION   object 
 1   ORTPLZ               float64
 2   SUM_INSURED          float64
 3   CONSTRACTION_DESIGN  object 
 4   CONSTRUCTION_YEAR    float64
 5   WFL                  float64
 6   ZONE                 object 
 7   DRAIN_PIPE_INSURED   int64  
 8   PRODUCTLINE          object 
 9   PRIOR_DAMAGES        int64  
 10  UVV-KZ               int64  
 11  UNDERWRITER          object 
 12  DAMAGE               int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 1.1+ GB
None
<class 'pandas.core.frame.DataFrame'>
Index: 8637107 entries, 0 to 11574438
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   CORPORATE_DEVISION   object 
 1   ORTPLZ               float64
 2   SUM_INSURED          float64
 3   CONSTRACTION_DESIGN  obj

In [6]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

print(features_binary.head())

    ORTPLZ  SUM_INSURED  CONSTRUCTION_YEAR    WFL  DRAIN_PIPE_INSURED  \
0  42109.0      71081.0        1967.565648   69.0                   0   
1  42277.0      55708.0        1967.565648   65.0                   0   
2  42389.0      74148.0        1967.565648   75.0                   0   
4  42553.0      75682.0        1967.565648  119.0                   0   
5  42113.0      70000.0        1967.565648  100.0                   0   

   PRIOR_DAMAGES  UVV-KZ  DAMAGE  CORPORATE_DEVISION_VHV  \
0              0       1       0                    True   
1              0       1       0                    True   
2              0       1       0                    True   
4              0       1       0                    True   
5              0       1       0                    True   

   CONSTRACTION_DESIGN_CARAVAN_MOTORHOME  ...  ZONE_4  ZONE_4.0  ZONE_5  \
0                                  False  ...   False     False   False   
1                                  False  ...   Fa

## Random Forest

In [64]:
# Define feature and target

X = features_binary.drop(columns=['DAMAGE'])
y = features_binary['DAMAGE']

# Split into test and training

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomUnderSampler
under_sampler = RandomUnderSampler(random_state=42)

# Resample the training data
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

In [65]:
# Initializing Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)

# Training the classifier
rf_classifier.fit(X_resampled, y_resampled)

# Making predictions
y_pred = rf_classifier.predict(X_test)



In [66]:
# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generating confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Displaying confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

# With undersampling accuracy gets worse, but we detect more TRUE Positives

Accuracy: 0.8019152239580137
Confusion Matrix:
[[1367244  335959]
 [   6217   18002]]
True Positives: 18002
False Positives: 335959
True Negatives: 1367244
False Negatives: 6217


In [16]:
# Feature-importnace analysis
feature_importance = rf_classifier.feature_importances_

# Display results
print("Feature Importance:")
for i, feature in enumerate(X_train.columns):
    print(f"{feature}: {feature_importance[i]}")

Feature Importance:
ORTPLZ: 0.2507575329769636
SUM_INSURED: 0.5560879080530164
CONSTRUCTION_YEAR: 0.08898752603786106
WFL: 0.03954166754507387
DRAIN_PIPE_INSURED: 0.0
PRIOR_DAMAGES: 0.004210705766032088
UVV-KZ: 0.0008838678151869576
CORPORATE_DEVISION_VHV: 0.012024849220961469
CONSTRACTION_DESIGN_CARAVAN_MOTORHOME: 7.458628197756248e-07
CONSTRACTION_DESIGN_DESIGN_CLASS_I: 1.5027999825769787e-05
CONSTRACTION_DESIGN_DESIGN_CLASS_II: 9.14955924222133e-09
CONSTRACTION_DESIGN_DESIGN_CLASS_III: 2.677595575158976e-06
CONSTRACTION_DESIGN_DESIGN_CLASS_IV: 4.091068684765485e-09
CONSTRACTION_DESIGN_DESIGN_CLASS_V: 4.920965314234112e-09
CONSTRACTION_DESIGN_NORMAL_VENTURE: 0.0002823402571098616
CONSTRACTION_DESIGN_PREDOMINANTLY_WOODEN_ROOF: 1.3356443086890538e-05
CONSTRACTION_DESIGN_PREFAB_HOUSE: 0.00022683551057533363
CONSTRACTION_DESIGN_PREFAB_HOUSE_I: 3.488368327384191e-10
PRODUCTLINE_Kompakt: 0.0023516388783437884
PRODUCTLINE_Plus: 0.0010074467990372077
PRODUCTLINE_Premium: 0.001282165117171353

## Random Forest with Rolling window

In [23]:
#Choose variables where it makes logical including year column

initial_features = contract_class.loc[:, ['CORPORATE_DEVISION','ORTPLZ',  'SUM_INSURED', 'CONSTRACTION_DESIGN','CONSTRUCTION_YEAR','WFL','ZONE','DRAIN_PIPE_INSURED','PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER','DAMAGE', 'YEAR']]

initial_features_clean = initial_features.dropna()

In [48]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

print(features_binary.info())

<class 'pandas.core.frame.DataFrame'>
Index: 8637107 entries, 0 to 11574438
Data columns (total 52 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   ORTPLZ                                         float64
 1   SUM_INSURED                                    float64
 2   CONSTRUCTION_YEAR                              float64
 3   WFL                                            float64
 4   DRAIN_PIPE_INSURED                             int64  
 5   PRIOR_DAMAGES                                  int64  
 6   UVV-KZ                                         int64  
 7   DAMAGE                                         int64  
 8   YEAR                                           int64  
 9   CORPORATE_DEVISION_VHV                         bool   
 10  CONSTRACTION_DESIGN_CARAVAN_MOTORHOME          bool   
 11  CONSTRACTION_DESIGN_DESIGN_CLASS_I             bool   
 12  CONSTRACTION_DESIGN_DESIGN_CLASS_II           

In [67]:
# Define feature and target

X = features_binary.drop(columns=['DAMAGE'])
y = features_binary['DAMAGE']

YEAR_INDEX = features_binary.columns.get_loc('YEAR')
print(YEAR_INDEX)

8


In [68]:

YEAR_INDEX = 8

# Iterate over unique years in the dataset
unique_years = np.unique(X.iloc[:, YEAR_INDEX])
yearly_accuracies = {}
for i in range(1, len(unique_years)):
    train_year = unique_years[i-1]
    test_year = unique_years[i]

    # Select data for training and testing
    X_train = X[X.iloc[:, YEAR_INDEX] == train_year]
    y_train = y[X.iloc[:, YEAR_INDEX] == train_year]
    X_test = X[X.iloc[:, YEAR_INDEX] == test_year]
    y_test = y[X.iloc[:, YEAR_INDEX] == test_year]

    # Undersample the training data
    under_sampler = RandomUnderSampler(random_state=42)
    X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

    # Initialize and train Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)
    rf_classifier.fit(X_resampled, y_resampled)

    # Make predictions
    y_pred = rf_classifier.predict(X_test)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Store accuracy value for the current year
    yearly_accuracies[test_year] = accuracy

    # Evaluate model
    print("Accuracy for year {}: {:.2f}".format(test_year, accuracy))

Accuracy for year True: 0.98


In [62]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generating confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Displaying confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

Accuracy: 0.9838248320000199
Confusion Matrix:
[[6315109   79791]
 [  24043     403]]
True Positives: 403
False Positives: 79791
True Negatives: 6315109
False Negatives: 24043
